<a href="https://colab.research.google.com/github/Fuohna/nlp-simple-chatbot-v1/blob/main/intent_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U tensorflow
!pip install -U tensorflow-text

In [17]:
import pandas as pd
import json
import string
import tensorflow as tf
import tensorflow_text as tf_text
import keras
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Preprocessing

In [18]:
train_data = "/content/drive/MyDrive/Colab Notebooks/NLP-Chatbot/train_dataset.csv"
train_df = pd.read_csv(train_data)
train_df

,domain,intent,utterance,bio
0,Airconditioner,TurnOff,switch the air conditioning off,o o b-av_alias i-av_alias o
1,Airconditioner,TurnOff,turn the ac off,o o b-av_alias o
2,Airconditioner,TurnOn,switch on the aircon,o o o b-av_alias
3,Airconditioner,TurnOn,turn the aircon on,o o b-av_alias o
4,Airconditioner,ChangeTemperature,change the minimum and maximum temperatures on...,o o o o o o o o o
...,...,...,...,...
1384,Youtube,ShowFollowing,show me channels I follow on youtube,o o o o o o o
1385,Youtube,ShowFollowing,show me youtube channels i follow,o o o o o o
1386,Youtube,ShowFollowing,show youtube channels i follow,o o o o o
1387,Youtube,ShowSubscribedChannels,show channels I am subscribed to on youtube,o o o o o o o o


In [19]:
texts = train_df['utterance'].apply(lambda sequence: [ltrs.lower() for ltrs in sequence if ltrs not in string.punctuation])
texts = texts.apply(lambda word: ''.join(word))

In [20]:
from tensorflow.keras.preprocessing.text import Tokenizer
from nltk.tokenize import word_tokenize 
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [21]:
word_list = texts.apply(lambda sen: word_tokenize(sen))
print(word_list)

0                   [switch, the, air, conditioning, off]
1                                    [turn, the, ac, off]
2                               [switch, on, the, aircon]
3                                 [turn, the, aircon, on]
4       [change, the, minimum, and, maximum, temperatu...
                              ...                        
1384         [show, me, channels, i, follow, on, youtube]
1385             [show, me, youtube, channels, i, follow]
1386                 [show, youtube, channels, i, follow]
1387    [show, channels, i, am, subscribed, to, on, yo...
1388    [show, me, channels, i, am, subscribed, to, on...
Name: utterance, Length: 1389, dtype: object


In [22]:
vocab_size = 20000
def tokenizer(words):
  token = Tokenizer(num_words = vocab_size)
  token.fit_on_texts(words)
  return token

In [23]:
def max_length(words):
  return(len(max(words, key = len)))

In [24]:
token = tokenizer(word_list)
seqs = token.texts_to_sequences(word_list)
utterance_2_seq = pad_sequences(seqs, maxlen= max_length(word_list)) 
print(utterance_2_seq.shape)
print(utterance_2_seq) 

(1389, 24)
[[  0   0   0 ... 325 326 126]
 [  0   0   0 ...   3 327 126]
 [  0   0   0 ...   2   3 273]
 ...
 [  0   0   0 ...  65  14 192]
 [  0   0   0 ...   4   2  19]
 [  0   0   0 ...   4   2  19]]


In [25]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler().fit(utterance_2_seq)
utt_features = list(scaler.fit_transform(utterance_2_seq))
print(type(utt_features))

<class 'list'>


In [26]:
train_df['utterance'] = utt_features
train_df['utterance']

0       [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
1       [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
2       [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
3       [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
4       [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
                              ...                        
1384    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
1385    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
1386    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
1387    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
1388    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
Name: utterance, Length: 1389, dtype: object

In [27]:
from sklearn.preprocessing import LabelEncoder

intent_list = list(train_df['intent'])
label = LabelEncoder().fit_transform(intent_list)
train_df['intent'] = label
train_df['intent']

0       173
1       173
2       174
3       174
4        10
       ... 
1384    145
1385    145
1386    145
1387    158
1388    158
Name: intent, Length: 1389, dtype: int64

In [ ]:
train_df.drop(["domain", "bio"], axis = 1, inplace = True)
# print(train_df)

In [60]:
sorted_df = train_df.sort_values(by= "intent", ignore_index= "True")
sorted_df

,intent,utterance
0,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
...,...,...
1384,179,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1385,179,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1386,179,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1387,179,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [47]:
def countX(list, X):
  return list.count(X)

In [181]:
from scipy.linalg.basic import levinson
def _train_test_split(train_size):
  Xy_train = pd.DataFrame()
  Xy_test = pd.DataFrame()
  list_utt = list(sorted_df['utterance'])
  list_intent = list(sorted_df['intent'])

  for intent in list_intent:
    len = countX(list_intent, intent)
    # print('len',len)
    tmp_intent = intent
    index = list_intent.index(intent)

    for i in range(0, int(len*train_size)):
      if (tmp_intent == list_intent[i+index]):
        # print(tmp_intent)
        tmp_df = pd.DataFrame([[list_intent[i+index], list_utt[i+index]]], columns = ['intent', 'utterance'])
        Xy_train = Xy_train.append(tmp_df, ignore_index = True)
        Xy_test = Xy_test.append(pd.DataFrame([[list_intent[index+len-1-i], list_utt[index+len-1-i]]], columns = ['intent', 'utterance']), ignore_index = True)
      else:
        break

  X_train = list(Xy_train['utterance'])
  y_train = list(Xy_train['intent'])
  X_test = list(Xy_test['utterance'])
  y_test = list(Xy_test['intent'])
  # X_test = sorted_df.drop(X_train)
  return X_train, y_train, X_test, y_test

In [179]:
# _train_test_split(0.8)

In [182]:
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn import metrics

# X_train, X_test, y_train, y_test = train_test_split(list(sorted_df['utterance']), list(sorted_df['intent']), train_size=0.8, random_state = 40)
X_train, y_train, X_test, y_test = _train_test_split(train_size = 0.8)

In [183]:
gnb = GaussianNB()
gnb.fit(X_train, y_train)
y_pred = gnb.predict(X_test)

print("Gaussian Naive Bayes model accuracy:", metrics.accuracy_score(y_test, y_pred))

Gaussian Naive Bayes model accuracy: 0.37955410549211527


In [184]:
from sklearn.ensemble import RandomForestClassifier

clf=RandomForestClassifier()
clf.fit(X_train, y_train);
prediction = clf.predict(X_test);
print(metrics.accuracy_score(y_test, prediction))

0.9616639477977161


In [185]:
from sklearn.svm import SVC

clfsvc = SVC(kernel='rbf', gamma=0.01, C=1000)
clfsvc.fit(X_train, y_train)

prediction = clfsvc.predict(X_test)
print(metrics.accuracy_score(y_test, prediction))

0.7091897770527461


In [186]:
from sklearn.tree import DecisionTreeClassifier

